In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers.agents import HfEngine, ReactCodeAgent

In [2]:
import HF_TOKEN
hf_token = HF_TOKEN.HF_TOKEN

In [3]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token = hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token = hf_token,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
from transformers import StoppingCriteria,TextStreamer

# Custom implementation of a stopping criteria
class MyStoppingCriteria(StoppingCriteria):
  def __init__(self, target_sequence, prompt):
      self.target_sequence = target_sequence
      self.prompt=prompt
      self.last=None
      self.new_token=''
  def __call__(self, input_ids, scores, **kwargs):
      # Get the generated text as a string

      generated_text = tokenizer.decode(input_ids[0])
      if self.last:
        self.new_token = self.new_token+ generated_text.replace(self.last,'')

      # Check if the target sequence appears in the generated text
      if self.target_sequence in self.new_token:


          return True  # Stop generation
      self.last=generated_text
      return False  # Continue generation

  def __len__(self):
      return 1

  def __iter__(self):
      yield self

In [6]:
# declare prompt_iteration variable as a global variable
# global prompt_iteration
prompt_iteration = 0
message_log_file = "/home/vp899/projects/llm_agent/Output/message_log.txt"
result_log_file = "/home/vp899/projects/llm_agent/Output/result_log.txt"
def llm_engine(messages, stop_sequences):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    global prompt_iteration
    prompt_iteration += 1
    # prompt_iteration = 0
    iteration_message = f"Iteration **** {prompt_iteration} ****"
    # append the iteration message and messages to /home/vp899/projects/llama3_ft1/Output/message_log.txt
    with open(message_log_file, "a") as f:
        f.write(f"{iteration_message}\n")
        for message in messages:
            f.write(f"{message}\n")
    terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    outputs = model.generate(input_ids, max_new_tokens=1024, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,stopping_criteria=MyStoppingCriteria("Observation:", messages))
    response = outputs[0][input_ids.shape[-1]:]
    output = tokenizer.decode(response, skip_special_tokens=True)
    # write the interation message and output to /home/vp899/projects/llama3_ft1/Output/result_log.txt
    with open(result_log_file, "a") as f:
        f.write(f"{iteration_message}\n")
        f.write(f"{output}\n")
    print(output)
    return output

In [7]:
agent = ReactCodeAgent(
    tools=[],
    llm_engine=llm_engine,
    additional_authorized_imports=["numpy", "pandas", "matplotlib.pyplot", "seaborn", "json"],
    max_iterations=10,
)

In [8]:
agent_user_prompt =   """
You are provided an input csv file. The details about the columns in the csv file shown in json format between triple quotes. 
'''
{
  "Line_Item": "INPUT",
  "SubCategory": "INPUT",
  "Part_Number": "INPUT",
  "Part_Description": "INPUT",
  "Plant__Location": "INPUT",
  "Plant_Country": "INPUT",
  "Plant_Region": "INPUT",
  "Annualized_Quantity": "INPUT",
  "Quoted_Currency": "REQUIRED",
  "Final_Unit_Price_to_CNHi_Ex_Works": "REQUIRED",
  "Final_Unit_Price_to_CNHi_with_CNHi_Preferred_Incoterms": "REQUIRED",
  "CNHi_Preferred_Incoterms": "REQUIRED",
  "Description_of_Raw_Materials": "PREFERRED",
  "Total_Cost_of_ALL_Raw_Materials_Per_Unit": "PREFERRED",
  "Welding_Method_Used": "PREFERRED",
  "Cost_of_3rd_Party_Purchased_SubComponents": "PREFERRED",
  "Cost_of_Packaging_Per_Unit": "REQUIRED",
  "Tariff_Cost__Section_301_Subcomponents_Only__Per_Unit_from_CN_to_US_ONLY": "PREFERRED",
  "Manufacturing_Location_City": "REQUIRED",
  "Manufacturing_Location_Country": "REQUIRED",
  "Shipping_Origin_City": "REQUIRED",
  "Shipping_Origin_Country": "REQUIRED",
  "Port_Location__Full_Address_International_Shipments_Only": null,
  "Minimum_Order_Quantity": "PREFERRED",
  "Production_Lead_Times_Calendar_Days": "PREFERRED",
  "Total_Tooling_Price": "PREFERRED",
  "Tooling_Lead_Time_Calendar_Days": "PREFERRED",
  "Tool_Description": "PREFERRED",
  "NonRecurring_Engineering_Expenses": "PREFERRED",
  "NonRecurring_Engineering_Expense_Description": "PREFERRED",
  "Reason_for_No_Quote": "PREFERRED",
  "Supplier_Comments": "PREFERRED"
}
'''
The columns are organized into sections. \nEach section has a list of column names and a condition. The condition can be either “REQUIRED”, “PREFERRED” or “INPUT”. \nThe “REQUIRED” columns are mandatory for the supplier to provide a value. The “PREFERRED” columns are optional for the supplier to provide a value. The “INPUT” columns are the input data that you will be provided. \n You can parse the json to understand the condition for each column. \nThe quote is provided in the “Total_Price” section. \nYou can consider a line item as quoted by the supplier if at least one REQUIRED column_name has not null value. Therefore the total line items quoted by the supplier is the count of all the rows with at least one REQUIRED column with a not null value. Line items quoted by supplier is an important information. The supplier has provided a quote for a line item if any one of the required columns have a not null value. Single space or “-“ must be considered as null. Therefore, you must replace required column values equal to single space or "-" or "_" to null.  \nYour job is to identify and report line items that are “Quoted by Supplier” and have null values in any of the required column_names. \n Your analysis can be considered as complete when you have reviewed all the required column_names for missing values. Once your analysis is complete, you can provide your final answer. \nAs final answer, you must present the list of violations by column_name. An example of a final answer is : \n1) Quoted_Currency: Out of 120 line items quoted by supplier, 50 lines have missing values \n2) Shipping_Origin_City: Out of the 120 line items quoted by supplier, 25 have missing values.\n You must aggregate return the final answer as an argument to the final_answer tool. For example final_answer(final answer)
"""

In [9]:
agent_user_prompt1 =   """
You are provided an input csv file. The details about the columns in the csv file shown in json format between triple quotes. 
'''
{
  "Line_Item": "INPUT",
  "SubCategory": "INPUT",
  "Part_Number": "INPUT",
  "Part_Description": "INPUT",
  "Plant__Location": "INPUT",
  "Plant_Country": "INPUT",
  "Plant_Region": "INPUT",
  "Annualized_Quantity": "INPUT",
  "Quoted_Currency": "REQUIRED",
  "Final_Unit_Price_to_CNHi_Ex_Works": "REQUIRED",
  "Final_Unit_Price_to_CNHi_with_CNHi_Preferred_Incoterms": "REQUIRED",
  "CNHi_Preferred_Incoterms": "REQUIRED",
  "Description_of_Raw_Materials": "PREFERRED",
  "Total_Cost_of_ALL_Raw_Materials_Per_Unit": "PREFERRED",
  "Welding_Method_Used": "PREFERRED",
  "Cost_of_3rd_Party_Purchased_SubComponents": "PREFERRED",
  "Cost_of_Packaging_Per_Unit": "REQUIRED",
  "Tariff_Cost__Section_301_Subcomponents_Only__Per_Unit_from_CN_to_US_ONLY": "PREFERRED",
  "Manufacturing_Location_City": "REQUIRED",
  "Manufacturing_Location_Country": "REQUIRED",
  "Shipping_Origin_City": "REQUIRED",
  "Shipping_Origin_Country": "REQUIRED",
  "Port_Location__Full_Address_International_Shipments_Only": null,
  "Minimum_Order_Quantity": "PREFERRED",
  "Production_Lead_Times_Calendar_Days": "PREFERRED",
  "Total_Tooling_Price": "PREFERRED",
  "Tooling_Lead_Time_Calendar_Days": "PREFERRED",
  "Tool_Description": "PREFERRED",
  "NonRecurring_Engineering_Expenses": "PREFERRED",
  "NonRecurring_Engineering_Expense_Description": "PREFERRED",
  "Reason_for_No_Quote": "PREFERRED",
  "Supplier_Comments": "PREFERRED"
}
'''
The columns are organized into sections. \nEach section has a list of column names and a condition. The condition can be either “REQUIRED”, “PREFERRED” or “INPUT”. \nThe “REQUIRED” columns are mandatory for the supplier to provide a value. The “PREFERRED” columns are optional for the supplier to provide a value. The “INPUT” columns are the input data that you will be provided. \n You can parse the json to understand the condition for each column. \nThe quote is provided in the “Total_Price” section. \nYou can consider a line item as quoted by the supplier if at least one REQUIRED column_name has not null value. Therefore the total line items quoted by the supplier is the count of all the rows with at least one REQUIRED column with a not null value. Line items quoted by supplier is an important information. The supplier has provided a quote for a line item if any one of the required columns have a not null value. Single space or “-“ must be considered as null. Therefore, you must replace required column values equal to single space or "-" or "_" to null.  \nYour job is to identify and report line items that are “Quoted by Supplier” and have null values in any of the required column_names. \n Your analysis can be considered as complete when you have reviewed all the required column_names for missing values. Once your analysis is complete, you can provide your final answer. \nYour final answer must have a statement about each column_name with missing values. An example of a final answer is : \n1) Quoted_Currency: Out of 120 line items quoted by supplier, 50 lines have missing values \n2) Shipping_Origin_City: Out of the 120 line items quoted by supplier, 25 have missing values.\n You must aggregate the final anwer statements and return the final answer as an argument to the final_answer tool. For example final_answer(final answer statements)
"""

In [10]:
analysis = agent.run(
    agent_user_prompt1,
    # additional_notes=data_notes,
    #source_file="/home/vp899/projects/llama3_ft1/Input/AG_Cylinder_Function.csv",
    source_file = "/home/vp899/projects/llm_agent/Input/BV_Dataset.csv",
)

======== New task ========

You are provided an input csv file. The details about the columns in the csv file shown in json format between triple quotes. 
'''
{
  "Line_Item": "INPUT",
  "SubCategory": "INPUT",
  "Part_Number": "INPUT",
  "Part_Description": "INPUT",
  "Plant__Location": "INPUT",
  "Plant_Country": "INPUT",
  "Plant_Region": "INPUT",
  "Annualized_Quantity": "INPUT",
  "Quoted_Currency": "REQUIRED",
  "Final_Unit_Price_to_CNHi_Ex_Works": "REQUIRED",
  "Final_Unit_Price_to_CNHi_with_CNHi_Preferred_Incoterms": "REQUIRED",
  "CNHi_Preferred_Incoterms": "REQUIRED",
  "Description_of_Raw_Materials": "PREFERRED",
  "Total_Cost_of_ALL_Raw_Materials_Per_Unit": "PREFERRED",
  "Welding_Method_Used": "PREFERRED",
  "Cost_of_3rd_Party_Purchased_SubComponents": "PREFERRED",
  "Cost_of_Packaging_Per_Unit": "REQUIRED",
  "Tariff_Cost__Section_301_Subcomponents_Only__Per_Unit_from_CN_to_US_ONLY": "PREFERRED",
  "Manufacturing_Location_City": "REQUIRED",
  "Manufacturing_Location_Count

Thought: I will first read the csv file using pandas and then parse the json to understand the condition for each column. I will then replace the required column values equal to single space or "-" or "_" to null. After that, I will identify and report line items that are “Quoted by Supplier” and have null values in any of the required column_names.

Code:
```py
import pandas as pd
import json

# Read the csv file
df = pd.read_csv('/home/vp899/projects/llm_agent/Input/BV_Dataset.csv')

# Parse the json to understand the condition for each column
conditions = json.loads('''
{
  "Line_Item": "INPUT",
  "SubCategory": "INPUT",
  "Part_Number": "INPUT",
  "Part_Description": "INPUT",
  "Plant__Location": "INPUT",
  "Plant_Country": "INPUT",
  "Plant_Region": "INPUT",
  "Annualized_Quantity": "INPUT",
  "Quoted_Currency": "REQUIRED",
  "Final_Unit_Price_to_CNHi_Ex_Works": "REQUIRED",
  "Final_Unit_Price_to_CNHi_with_CNHi_Preferred_Incoterms": "REQUIRED",
  "CNHi_Preferred_Incoterms": "REQUI

==== Agent is executing the code below:
# Count the number of line items that are “Quoted by Supplier” and have null values in any of the required column_names
quoted_null_items = quoted_items[required_columns].isnull().sum().to_frame('count')
unquoted_null_items = unquoted_items[required_columns].isnull().sum().to_frame('count')
print(quoted_null_items)
print(unquoted_null_items)
====
Print outputs:
                                                    count
Quoted_Currency                                       467
Final_Unit_Price_to_CNHi_Ex_Works                     469
Final_Unit_Price_to_CNHi_with_CNHi_Preferred_In...    649
CNHi_Preferred_Incoterms                              649
Cost_of_Packaging_Per_Unit                            469
Manufacturing_Location_City                           469
Manufacturing_Location_Country                          0
Shipping_Origin_City                                  469
Shipping_Origin_Country                               469
                

Thought: Now I will count the number of line items that are “Quoted by Supplier” and have null values in any of the required column_names.

Code:
```py
# Count the number of line items that are “Quoted by Supplier” and have null values in any of the required column_names
quoted_null_items = quoted_items[required_columns].isnull().sum().to_frame('count')
unquoted_null_items = unquoted_items[required_columns].isnull().sum().to_frame('count')
print(quoted_null_items)
print(unquoted_null_items)
```<end_action>

Observation:



==== Agent is executing the code below:
# Create the final answer statements
final_answer_statements = []
for col, count in quoted_null_items.iterrows():
    final_answer_statements.append(f'{col}: Out of {len(quoted_items)} line items quoted by supplier, {count["count"]} lines have missing values')
for col, count in unquoted_null_items.iterrows():
    final_answer_statements.append(f'{col}: Out of {len(unquoted_items)} line items unquoted by supplier, {count["count"]} lines have missing values')
final_answer_statements = '\n'.join(final_answer_statements)
final_answer(final_answer_statements)
====
Print outputs:

>>> Final answer:
Quoted_Currency: Out of 649 line items quoted by supplier, 467 lines have missing values
Final_Unit_Price_to_CNHi_Ex_Works: Out of 649 line items quoted by supplier, 469 lines have missing values
Final_Unit_Price_to_CNHi_with_CNHi_Preferred_Incoterms: Out of 649 line items quoted by supplier, 649 lines have missing values
CNHi_Preferred_Incoterms: Out of 649

Thought: Now I will create the final answer statements.

Code:
```py
# Create the final answer statements
final_answer_statements = []
for col, count in quoted_null_items.iterrows():
    final_answer_statements.append(f'{col}: Out of {len(quoted_items)} line items quoted by supplier, {count["count"]} lines have missing values')
for col, count in unquoted_null_items.iterrows():
    final_answer_statements.append(f'{col}: Out of {len(unquoted_items)} line items unquoted by supplier, {count["count"]} lines have missing values')
final_answer_statements = '\n'.join(final_answer_statements)
final_answer(final_answer_statements)
```<end_action>


In [11]:
print(analysis)

Quoted_Currency: Out of 649 line items quoted by supplier, 467 lines have missing values
Final_Unit_Price_to_CNHi_Ex_Works: Out of 649 line items quoted by supplier, 469 lines have missing values
Final_Unit_Price_to_CNHi_with_CNHi_Preferred_Incoterms: Out of 649 line items quoted by supplier, 649 lines have missing values
CNHi_Preferred_Incoterms: Out of 649 line items quoted by supplier, 649 lines have missing values
Cost_of_Packaging_Per_Unit: Out of 649 line items quoted by supplier, 469 lines have missing values
Manufacturing_Location_City: Out of 649 line items quoted by supplier, 469 lines have missing values
Manufacturing_Location_Country: Out of 649 line items quoted by supplier, 0 lines have missing values
Shipping_Origin_City: Out of 649 line items quoted by supplier, 469 lines have missing values
Shipping_Origin_Country: Out of 649 line items quoted by supplier, 469 lines have missing values
Quoted_Currency: Out of 0 line items unquoted by supplier, 0 lines have missing valu